In [51]:
from bycon import *

In [49]:
mapfile = read_tsv_to_dictlist('../temp/mappingfile.tsv')
mapfile = mapfile[0]

In [50]:
mapfile

[{'biosample_id': 'pgxbs-kftvhxry',
  'variant_id': 'pgxvar-',
  'callset_id': 'pgxcs-lhkcspq7',
  'chromosome': 'chr1',
  'start': '92246555',
  'end': '92246555',
  'strand': '+',
  'reference_bases': 'G',
  'alternate_bases': 'C',
  'hgvsc': 'c.1760C>G',
  'variant_classification': 'Missense_Mutation',
  'variant_type': 'SNP',
  'hgvsp': 'p.Ser587Cys',
  'hgvsp_short': 'p.S587C',
  'aliquot_id': 'd7593b2a-0d86-44aa-a404-7fd1b10f65d4',
  'reference_id': '6934dbbc-19e6-42c3-9e9f-1db3e7b7981d',
  'case_id': 'pgx:TCGA-edc62c30-cacc-457d-96ba-d05ed35432a3',
  'sample_id': 'pgx:TCGA-1e6b9fc4-3fed-4d32-89b8-aa15fb4a5840'},
 {'biosample_id': 'pgxbs-kftvhxry',
  'variant_id': 'pgxvar-',
  'callset_id': 'pgxcs-lhkcspq7',
  'chromosome': 'chr1',
  'start': '226064454',
  'end': '226064454',
  'strand': '+',
  'reference_bases': 'G',
  'alternate_bases': 'C',
  'hgvsc': 'c.103G>C',
  'variant_classification': 'Missense_Mutation',
  'variant_type': 'SNP',
  'hgvsp': 'p.Gly35Arg',
  'hgvsp_short'

In [46]:
mapfile = mapfile[0:100]

In [47]:
for dic in mapfile:
    print(dic)

{'case_id': 'edc62c30-cacc-457d-96ba-d05ed35432a3', 'sample_id': '1e6b9fc4-3fed-4d32-89b8-aa15fb4a5840', 'aliquot_id': 'd7593b2a-0d86-44aa-a404-7fd1b10f65d4', 'reference_id': '6934dbbc-19e6-42c3-9e9f-1db3e7b7981d', 'chromosome': 'chr1', 'start': '92246555', 'end': '92246555', 'strand': '+', 'variant_classification': 'Missense_Mutation', 'variant_type': 'SNP', 'ref_allele': 'G', 'tumor_allele1': 'G', 'tumor_allele2': 'C', 'hgvsc': 'c.1760C>G', 'hgvsp': 'p.Ser587Cys', 'hgvsp_short': 'p.S587C'}
{'case_id': 'edc62c30-cacc-457d-96ba-d05ed35432a3', 'sample_id': '1e6b9fc4-3fed-4d32-89b8-aa15fb4a5840', 'aliquot_id': 'd7593b2a-0d86-44aa-a404-7fd1b10f65d4', 'reference_id': '6934dbbc-19e6-42c3-9e9f-1db3e7b7981d', 'chromosome': 'chr1', 'start': '226064454', 'end': '226064454', 'strand': '+', 'variant_classification': 'Missense_Mutation', 'variant_type': 'SNP', 'ref_allele': 'G', 'tumor_allele1': 'G', 'tumor_allele2': 'C', 'hgvsc': 'c.103G>C', 'hgvsp': 'p.Gly35Arg', 'hgvsp_short': 'p.G35R'}
{'case_

In [45]:
    mongo_client = MongoClient( )
    var_coll = MongoClient( )[ ds_id ][ "variants" ]

    bios_ids = set()
    for c, v in enumerate(variants):
        bs_id = v.get("biosample_id", False)
        if not bs_id:
            print("¡¡¡ The `biosample_id` parameter is required for variant assignment !!!")
            exit()
        if not "pgxbs-" in bs_id:
            print("¡¡¡ The `biosample_id` parameter has to start with 'pgxbs-' !!!")
            exit()
        bios_ids.add(bs_id)

    print("=> The variants file contains {} variants from {} samples.".format(var_no, len(bios_ids)))
    proceed = input("Do you want to continue to update database **{}**?\n(Y|n): ".format(ds_id))
    if "n" in proceed.lower():
        exit()


    if "y" in delBiosVars.lower():
        for b_del in bios_ids:
            v_d = var_coll.delete_many({"biosample_id": b_del})
            print("==>> deleted {} variants from {}".format(v_d.deleted_count, b_del))

    for c, v in enumerate(variants):

        bs_id = v.get("biosample_id", False)

        n = str(c+1)
 
        # TODO: This is a bit of a double definition; disentangle ...
        update_v = {
            "legacy_id": v.get("variant_id", "pgxvar-"+n),
            "biosample_id": bs_id,            
            "callset_id": v.get("callset_id", re.sub("pgxbs-", "pgxcs-", bs_id)),
            "individual_id": v.get("individual_id", re.sub("pgxbs-", "pgxind-", bs_id))
        }

        update_v = import_datatable_dict_line(byc, update_v, fieldnames, v, "variant")
        update_v.update({
            "variant_internal_id": variant_create_digest(update_v, byc),
            "updated": datetime.datetime.now().isoformat()
        })

        if not byc["test_mode"]:
            vid = var_coll.insert_one( update_v  ).inserted_id
            vstr = 'pgxvar-'+str(vid)
            var_coll.update_one({'_id':vid},{'$set':{ 'id':vstr }})
            print("=> inserted {}".format(vstr))
        else:
            prjsonnice(update_v)

        # prjsonnice(update_v)



2570090